[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/tagging.ipynb)

# 将文本分类为标签

标记是指为文档打上诸如以下之类的类别标签：

- 情绪
- 语言
- 风格（例如，正式、非正式）
- 涵盖的主题
- 政治倾向

![Image description](../../static/img/tagging.png)

## 概述

标记包含几个组件：

* `function`：与 [提取](/docs/tutorials/extraction) 类似，标记也使用 [函数](https://openai.com/blog/function-calling-and-other-api-updates) 来指定模型应如何标记文档
* `schema`：定义我们希望如何标记文档

## 入门

让我们通过一个非常简单的示例，了解如何在 LangChain 中使用 OpenAI 工具调用进行标记。我们将使用 OpenAI 模型支持的 [`with_structured_output`](/docs/how_to/structured_output) 方法。

In [ ]:
pip install --upgrade --quiet langchain-core

我们需要加载一个 [chat model](/docs/integrations/chat/)：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

让我们用 Pydantic 模型指定具有几个属性及其在 schema 中的预期类型。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

如果我们想要字典输出，可以直接调用 `.model_dump()`

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

{'sentiment': 'enojado', 'aggressiveness': 8, 'language': 'es'}

正如我们在示例中看到的，它正确地解释了我们的意图。

结果各不相同，因此我们可能会得到不同语言的情绪（例如，“积极”、“enojado”等）。

我们将在下一节中了解如何控制这些结果。

## 更精细化的控制

精心的模式定义让我们能够更精细化地控制模型的输出。

具体来说，我们可以定义：

- 每个属性的可能值
- 描述，以确保模型理解该属性
- 模型必须返回的必需属性

让我们重新声明我们的 Pydantic 模型，以 enum 的方式控制前面提到的各个方面：

In [11]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [15]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)

现在答案将以我们期望的方式受到限制！

In [12]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [13]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='enojado', aggressiveness=8, language='es')

In [14]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='neutral', aggressiveness=1, language='English')

LangSmith 的[追踪](https://smith.langchain.com/public/38294e04-33d8-4c5a-ae92-c2fe68be8332/r)让我们得以窥探其内部运作：

![Image description](../../static/img/tagging_trace.png)

### 深入了解

* 您可以使用 [metadata tagger](/docs/integrations/document_transformers/openai_metadata_tagger) 文档转换器从 LangChain `Document` 中提取元数据。
* 这涵盖了与 tagging chain 基本相同的功能，只是应用于 LangChain `Document`。